In [1]:
import os
import pandas as pd
from tkinter.filedialog import askopenfilename
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erf, erfinv # Modified code to include ionization factor (N^2/3)
from scipy.optimize import curve_fit
from math import exp,sqrt, log, sinh, cosh, floor
from pathlib import Path
import Deflection_Lib as funcs #File containing some helper functions
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets

In [48]:
'''
undeflected_data = widgets.FileUpload(multiple=False)
deflected_data = widgets.FileUpload(multiple=False)
undeflected_data_hbox = widgets.HBox([widgets.Label(value="Upload Undeflected Data: "),undeflected_data])
deflected_data_hbox = widgets.HBox([widgets.Label(value="Upload Undeflected Data: "),deflected_data])

display(undeflected_data_hbox)
display(deflected_data_hbox)
'''

##Take PV fit to undeflected data
undefl_txt=askopenfilename(title="SELECT UNDEFLECTED DATA", initialfile="1_Fitted_Undeflected_Data")
undeflected_data = np.loadtxt(undefl_txt)

'''
undeflected_left_boundary = undeflected_data[0,0] #domain of zero-field profile
undeflected_right_boundary = undeflected_data[-1,0]
undeflected_data_x_vals = undeflected_data[:,0]
undeflected_data_y_vals = undeflected_data[:,1]
undeflected_data_max = max(undeflected_data_y_vals)
'''


##Take PV fit to deflected data, x-values will be used for simulation output
deflec_txt = askopenfilename(title="SELECT DEFLECTED DATA", initialfile= "2_Fitted_Deflected_Data")
deflected_data =  np.loadtxt(deflec_txt)
'''
deflected_data_x_vals = deflected_data[:,0]
deflected_data_y_vals = deflected_data[:,1]
deflected_sim_right_bound = deflected_data_x_vals[-1]
deflected_sim_left_bound = deflected_data_x_vals[0]

simulated_deflection_data_x_vals = deflected_data_x_vals  # Instantiate array for sim output's xvals
'''



In [50]:
class simulation_parameters:
    rat = 0.9 #0.649336  #ratio of standard deviation to average droplet size
    vx = 0.2 #velocity in the x-direction for a particular droplet; used by Droplet() & Deflection()
    vz = 0.0 #velocity in the z-direction for a particular droplet; used by Deflection()

    moleculeName = "Cl2Fe" ## Name of Molecule Used in Simulation
    mDop = 126.75*1.66E-27 ## Multiply to change from amu to kg
    pDop = 5.4             ## Magnetic moment of dopant (if known)
    TDop = 638             ## Temperature of the dopant in a given experiment

    v0 = 256          # From MCS data
    number_of_iterations = int(1e4)   #Max number of trials per position in zero field data # JN quintupled
    startAveN = 10000       # Reasonable first Guess

    maxAveN = 14000   # Upper limit for loop
    increment = 500   # Change in droplet size per loop
    loopLimit = int((maxAveN - startAveN)/increment+1)

    vibration_Boiloff = 1654






rat = widgets.Text(value='0.9',placeholder='0.9',description='rat',disabled=False)
vx = widgets.Text(value='0.0',placeholder='0.9',description='vx',disabled=False)
vz = widgets.Text(value='0.0',placeholder='0.9',description='vz',disabled=False)

moleculeName = widgets.Text(placeholder ='FeCl2', description='Molecule Name', disabled = False)
mDop = widgets.Text(placeholder = '126.75', description = 'mDop') ## IN AMU. Ignore this: Multiply to change from amu to kg
pDop = widgets.Text(placeholder = '5.4', description = 'pDop')             ## Magnetic moment of dopant (if known)
TDop = widgets.Text(placeholder = '638', description = 'TDop')             ## Temperature of the dopant in a given experiment

v0 = widgets.Text(value='256',placeholder='256',description='v0',disabled=False)           # From MCS data
number_of_iterations = widgets.Text(value='10000',placeholder='10000',description='number_of_iterations',disabled=False)#int(1e4)   #Max number of trials per position in zero field data # JN quintupled
startAveN = widgets.Text(value='',placeholder='6000',description='startAveN',disabled=False)       # Reasonable first Guess

maxAveN = widgets.Text(value='14000',placeholder='14000',description='maxAveN',disabled=False)   # Upper limit for loop
increment = widgets.Text(value='500',placeholder='500',description='increment',disabled=False)   # Change in droplet size per loop

vibration_Boiloff = widgets.Text(value='1654',placeholder='1654',description='Vibrational Boiloff',disabled=False)

#Button to save parameters
save_button = widgets.Button(
    description = 'Submit Parameters!',
    button_style='info',
    
)


def save_parameters(b):
    simulation_parameters.rat=float(rat.value)
    simulation_parameters.vx=float(vx.value)
    simulation_parameters.vz=float(vz.value)

    simulation_parameters.moleculeName = moleculeName.value
    simulation_parameters.mDop=float(mDop.value)*1.66E-27
    simulation_parameters.pDop=float(pDop.value)
    simulation_parameters.TDop=float(TDop.value)

    simulation_parameters.v0 = int(v0.value)
    simulation_parameters.number_of_iterations = int(number_of_iterations.value)
    simulation_parameters.startAveN = int(startAveN.value)

    simulation_parameters.maxAveN = maxAveN.value
    simulation_parameters.increment = increment.value

    simulation_parameters.vibration_Boiloff = vibration_Boiloff.value
style = {'description_width': 'initial'}

save_button.on_click(save_parameters)

#undefl_txt=widgets.Text(value=undeflected_data.value[0]["name"], description= "Undeflected Data Upload", disabled = True, style = style)
#display(undefl_txt)

#defl_txt=widgets.Text(value=deflected_data.value[0]["name"], description= "Deflected Data Upload", disabled = True, style = style)
#display(defl_txt)

display(rat)
display(vx)
display(vz)

display(moleculeName)
display(mDop)
display(pDop)
display(TDop)

display(v0)
display(number_of_iterations)
display(startAveN)

display(maxAveN)
display(increment)
display(vibration_Boiloff)

display(save_button)










TypeError: non-string returned while reading data

In [163]:
print("rat: ",simulation_parameters.rat)
print("vx: ",simulation_parameters.vx)
print("vz: ",simulation_parameters.vz)
print("Name: ",simulation_parameters.moleculeName)
print("mDop: ",simulation_parameters.mDop)
print("pDop: ",simulation_parameters.pDop)
print("TDop: ",simulation_parameters.TDop)
print("v0: ",simulation_parameters.v0)
print("number of iterations: ",simulation_parameters.number_of_iterations)
print("average start number: ",simulation_parameters.startAveN)
print("Max: ",simulation_parameters.maxAveN)
print("increment: ",simulation_parameters.increment)
print("vibrational boiloff: ",simulation_parameters.vibration_Boiloff)


rat:  0.9
vx:  0.0
vz:  0.0
Name:  fwefwe
mDop:  2.0438417999999999e-22
pDop:  1231.0
TDop:  1231.0
v0:  256
number of iterations:  10000
average start number:  213
Max:  14000
increment:  500
vibrational boiloff:  1654
